In [ ]:
from cil.framework import AcquisitionGeometry, ImageGeometry
from cil.io import NEXUSDataWriter
from cil.plugins.astra.processors import FBP
from cil.plugins.astra.operators import ProjectionOperator
from cil.utilities.display import show2D
from utils import download_zenodo

from cil.optimisation.algorithms import PDHG
from cil.optimisation.operators import BlockOperator, GradientOperator, ZeroOperator, FiniteDifferenceOperator, IdentityOperator
from cil.optimisation.functions import L2NormSquared, L1Norm, MixedL21Norm, BlockFunction, IndicatorBox, ZeroFunction

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import time

import os

**First we need to download the raw data files used for reconstruction from [Zenodo](https://zenodo.org/record/5825464). For the powder phantom, there are three main datasets:**

1) powder_phantom_180s_sinogram.mat (Matlab file for Scan A dataset of 180 projections, 180s exposure time. The dataset has already been flatfield corrected).

2) powder_phantom_30s_sinogram.mat (Matlab file for Scan B dataset of 30 projections, 30s exposure time. The dataset has already been flatfield corrected).

3) Energy_axis.mat (Matlab file providing the direct energy-channel conversion, useful for analysing reconstructed datasets at different channels or different energies).

This may take some time.  

**Note:** The `download_zenodo` function requires the `wget` python package to access Zenodo files. If you don't have it, you can install using the command `conda install -c conda-forge python-wget`.

**Note 2:** You can skip this part if you already downloaded the powder data from the accompanying script `Powder_Phantom_180s_180Proj_FDK.ipynb`.

In [ ]:
download_zenodo()

In [ ]:
#%% Read data for Scan B datasets
pathname = os.path.abspath("MatlabData/")

# Scan B dataset - 30s, 30 projections
datafile = "Powder_phantom_30s_30Proj_sinogram.mat"

path = os.path.join(pathname,datafile)

tmp_X = sio.loadmat(path)   
X = tmp_X['S_30_30']

# Read Energy-Channel conversion
tmp_energy_channels = sio.loadmat(pathname + "/Energy_axis.mat")
ekeV = tmp_energy_channels['E_axis']
ekeV_crop = ekeV[0][99:199]

Sinogram raw data shape is [Vertical, Angles, Horizontal, Channels].  
However we need it in the shape [Channels, Vertical, Angles, Horizontal].  
We reorder using `np.swapaxes`

In [ ]:
print('Original Shape: {}'.format(X.shape))
X = np.swapaxes(X, 0, 3)
X = np.swapaxes(X, 1, 2)
print('Reordered Shape: {}'.format(X.shape))

In [ ]:
#%% Crop and rotate data to match data in paper

X = X[99:199] # Crop data to reduced channel subset (channels 100-200)
X = np.transpose(X,(0,3,2,1)) # Rotate data
print('Reduced Shape: {}'.format(X.shape))

In [ ]:
#%% Data shape information
num_channels = X.shape[0]
horizontal = X.shape[3]
vertical = X.shape[1]
num_angles = X.shape[2]

angles = np.linspace(-180+45,180+45,num_angles,endpoint=False)*np.pi/180

In [ ]:
#%% Define imaging scan metadata

# Scan parameters
distance_source_center = 318.0  # [mm]
distance_center_detector = 492.0  # [mm]
detector_pixel_size = 0.25  # [mm]

In [ ]:
#%% Define AcquisitionGeometry from imaging scan parameters

ag = AcquisitionGeometry.create_Cone3D(source_position = [0,-distance_source_center,0],
                                       detector_position = [0,distance_center_detector,0])\
                                     .set_panel([horizontal,vertical],[detector_pixel_size,detector_pixel_size])\
                                     .set_channels(num_channels)\
                                     .set_angles(-angles,angle_unit="radian")\
                                     .set_labels(['channel', 'vertical', 'angle', 'horizontal'])

# Create the 4D acquisition data
data = ag.allocate()
data.fill(X)

print(data)

In [ ]:
# Get the ImageGeometry directly from the AcquisitionGeometry using ig = ag.get_ImageGeometry()

ig = ag.get_ImageGeometry()

In [ ]:
# Setup the tomography operator for 3D hyperspectral data using the AcquisitionGeometry and ImageGeometry

ag3D = ag.get_slice(channel=0)
ig3D = ag3D.get_ImageGeometry()

## FDK Reconstruction

In [ ]:
# Allocate space for the FBP_4D recon

FBP_recon_4D = ig.allocate()

t = time.time()
        
# FBP reconstruction per channel
for i in range(ig.channels):
    
    FBP_recon_3D = FBP(ig3D, ag3D, 'gpu')(data.get_slice(channel=i))
    FBP_recon_4D.fill(FBP_recon_3D, channel=i)
    
    print("Finish FBP recon for channel {}".format(i), end='\r')
    
print("\nFDK Reconstruction Complete!")
tot = time.time() - t
print('Runtime: {} s'.format(tot))

In [ ]:
# Test image

plt.imshow(FBP_recon_4D.as_array()[50,40,:,:],cmap='inferno',vmin=0.0,vmax=3.0)

In [ ]:
#%% Save as nxs file with NEXUSDataWriter

name = "Powder_30s_30Proj_FDK.nxs"
writer = NEXUSDataWriter(file_name = "HyperspectralData/" + name,
                         data = FBP_recon_4D)
writer.write()

## PDHG Reconstruction with Space TV and Channel TGV Regularisation

In [ ]:
#%% Set up AstraProjector for 3D Multi-channel dataset

A3DMC = ProjectionOperator(ig, ag, 'gpu')

In [ ]:
# Set up Block Operator for combined Space TV - Channel TGV regularisation

op11 = GradientOperator(ig, correlation='Space', backend = "numpy")
op12 = ZeroOperator(ig, op11.range_geometry())

op21 = FiniteDifferenceOperator(ig, direction = 0)
op22 = -IdentityOperator(ig)

op31 = ZeroOperator(ig)
op32 = FiniteDifferenceOperator(ig, direction = 0)

op41 = A3DMC
op42 = ZeroOperator(ig, ag)

operator = BlockOperator(op11, op12, 
                         op21, op22, 
                         op31, op32, 
                         op41, op42, shape=(4,2))

# Compute operator Norm
normK = operator.norm()

sigma = 1./normK
tau = 1./normK


## Set up and run PDHG TV-TGV algorithm

In [ ]:
# Set regularisation weighting parameters

alpha = 0.002
beta = 0.18
gamma = np.sqrt(2) * beta

In [ ]:
# Build BlockFunction

f1 = alpha * MixedL21Norm()
f2 = beta * L1Norm() 
f3 = gamma * L1Norm()
f4 = 0.5 * L2NormSquared(b=data)

f = BlockFunction(f1, f2, f3, f4)         
g = BlockFunction(IndicatorBox(lower=0),ZeroFunction())  


In [ ]:
# Run reconstruction algorithm for 1000 iterations

t = time.time()

# Run the PDHG algorithm
print(alpha, beta, gamma)  
pdhg = PDHG(f=f, g=g, operator=operator, tau=tau, sigma=sigma, 
            max_iteration = 2000 , update_objective_interval = 100)        
pdhg.run(1000, verbose = 1)
        
print('Finished!')
tot = time.time() - t
print('Runtime: {} s'.format(tot))

In [ ]:
# Test image

plt.imshow(pdhg.solution[0].as_array()[50,40,:,:],cmap='inferno',vmin=0.0,vmax=3.0)

In [ ]:
# Save result as nxs file with NEXUSDataWriter

name = "{}_iters_alpha_{}_beta_{}.nxs".format(pdhg.iteration,alpha,beta)
writer = NEXUSDataWriter(file_name="HyperspectralData/" + name,
                        data = pdhg.solution[0])
writer.write()